# Betting Using Prob_HDWPSRRating

In [1]:
import sys
sys.path.append('I:/YaoTony/code/x8313')
# sys.path.append('d:/Codes/EquineTrading/')
from datetime import date
import pytest
from pandas import date_range
import pandas as pd
from horse.betsim.wrap.jcapper import JCapper
from horse.betsim.math import compute_probs_from_odds
import numpy as np

In [106]:
df_calibrate = pd.read_csv("DF_CALIBRATE_20180629.csv")
df_calibrate.head()

D:\Softwares\Anaconda3\envs\py35\lib\site-packages\IPython\core\interactiveshell.py:2728: DtypeWarning: Columns (12,27,84,85,87,88,93) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,race_id,EX_payout,PL_payout,SH_payout,SU_payout,TR_payout,WN_payout,index,track_id,date,...,year,month,weekday,prob_x8runner_HDWPSRRating_norm_par,EX_pool,PL_pool,SH_pool,SU_pool,TR_pool,WN_pool
0,APX_20100625_1,6.6,2.15,1.633333,78.349995,24.1,2.05,0.0,APX,2010-06-25,...,2010.0,6.0,4.0,0.011273,51904.0,80078.0,80078.0,17933.0,42794.0,80078.0
1,APX_20100625_1,6.6,2.15,1.633333,78.349995,24.1,2.05,1.0,APX,2010-06-25,...,2010.0,6.0,4.0,0.332936,51904.0,80078.0,80078.0,17933.0,42794.0,80078.0
2,APX_20100625_1,6.6,2.15,1.633333,78.349995,24.1,2.05,2.0,APX,2010-06-25,...,2010.0,6.0,4.0,0.067706,51904.0,80078.0,80078.0,17933.0,42794.0,80078.0
3,APX_20100625_1,6.6,2.15,1.633333,78.349995,24.1,2.05,3.0,APX,2010-06-25,...,2010.0,6.0,4.0,0.026951,51904.0,80078.0,80078.0,17933.0,42794.0,80078.0
4,APX_20100625_1,6.6,2.15,1.633333,78.349995,24.1,2.05,4.0,APX,2010-06-25,...,2010.0,6.0,4.0,0.150379,51904.0,80078.0,80078.0,17933.0,42794.0,80078.0


In [175]:
df_calibrate = df_calibrate[df_calibrate.groupby("race_id")["race_id"].transform(lambda x: len(x))!=1]

In [176]:
bet_payout_dict = {"Win":"WN_payout","Exacta":"EX_payout"}

## Harville.py: Add "Win" and "Quinella" in harville_multiprob

In [180]:
import Harville

In [181]:
Harville.harville_multiprob([1,2,3],[0.2,0.39,0.41],bet_type = "Win")

[[3, 0.41], [2, 0.39], [1, 0.2]]

In [182]:
Harville.harville_multiprob([1,2,3],[0.2,0.39,0.41],bet_type = "Quinella")

[[2, 3, 0.5331480966935259],
 [1, 3, 0.24148305084745758],
 [1, 2, 0.22536885245901642]]

## Apply Harville model to guide Win and Exacta betting

In [202]:
def harville_betting(df, bet_type="Win"):
    df = df.dropna(axis=0)

    df = df.sort_values(by = ["race_id"])
    column_name = "{}_is_betted_rightly".format(bet_type)
    df[column_name] = False
    race_set = set(df['race_id'])
    for race_id in race_set:
        df_sub = df[df['race_id']==race_id]
        try:
            fav = Harville.harville_multiprob(\
                df_sub['runner_program_number_pp'].values,df_sub['prob_x8runner_HDWPSRRating_norm_par'].values,bet_type)[0]
            n_sequence = len(fav)-1
            is_correct = np.zeros(n_sequence)
            for i in range(n_sequence):
                is_correct[i] = (df_sub[df_sub["runner_program_number_pp"]==fav[i]]["official_finish_position"]==i+1).bool()

            if(is_correct.all()):
                df.loc[df["race_id"]==race_id,column_name] = True 
        
        # todo: some wierd error
        except:
            pass
#             print(df_sub)
        
        
    return df

In [187]:
def expected_return(payout_win,prob_win,rebate=0,bet=1):
    pnl = bet*(prob_win*payout_win + (1-prob_win)*(-1) + rebate)
    return pnl

In [206]:
def betting_test(df):
    is_right_columns = []
    bet_types = []
    payout_columns = []
    for column_name in df.columns:
        indx = column_name.find("_is_betted_rightly")
        if(indx!=-1):
            is_right_columns.append(column_name)
            bet_types.append(column_name[:indx])
            payout_columns.append(bet_payout_dict[column_name[:indx]])
    
    # Precesion Calculation
    for bet_type,is_correct in zip(bet_types,is_right_columns):
        prob_win = df[is_correct].value_counts(normalize=True)[1]
        print("the precision of {}: {}".format(bet_type, prob_win))
        
    # PnL Calculation
    race_set = set(df['race_id'])
    for bet_type,is_correct,bet_payout in zip(bet_types,is_right_columns,payout_columns):
        pnl = 0
        for race_id in race_set:
            df_sub = df[df['race_id']==race_id]
#             print(bet_payout)
#             print(df_sub[bet_payout])
            is_paid = int(df_sub[is_correct].all())
            payout = df_sub[bet_payout].iloc[0]
            pnl += expected_return(payout, is_paid, 0, 1)
        pnl/=len(race_set)
            
        print("the pnl for {}: {}".format(bet_type, pnl))

    pass

## Scenarios Comparison

### Cases

* TR pool > EX pool
* TR pool < EX pool

### Metrics

* Prediction Precision
* Actual History PnL (You might want to give me rebate rates)

### When TR pool > EX pool

In [209]:
df_calibrate_2 = df_calibrate[df_calibrate["TR_pool"]>df_calibrate["EX_pool"]]
df = df_calibrate_2[['race_id','EX_payout',"WN_payout",'runner_program_number_pp','official_finish_position','prob_x8runner_HDWPSRRating_norm_par']]
df = harville_betting(df,'Win')
df = harville_betting(df,"Exacta")

In [210]:
betting_test(df)

the precision of Win: 0.37425860023724794
the precision of Exacta: 0.08481613285883749
the pnl for Win: 0.4029288702928871
the pnl for Exacta: -0.16799163179916318


### When TR pool < EX pool

In [211]:
df_calibrate_3 = df_calibrate[df_calibrate["TR_pool"]<df_calibrate["EX_pool"]]
df = df_calibrate_3[['race_id','EX_payout',"WN_payout",'runner_program_number_pp','official_finish_position','prob_x8runner_HDWPSRRating_norm_par']]
df = harville_betting(df,'Win')
df = harville_betting(df,"Exacta")
betting_test(df)

the precision of Win: 0.3432620706221475
the precision of Exacta: 0.10076867643526304
the pnl for Win: 0.29129662522202476
the pnl for Exacta: 0.062344582593250486


# Conclusion

* The model performed better when TR pool > EX pool. 
* Betting on Win based on Prob_HDWPSRRating_norm_par is much more benificial and accurate(stable) than betting on Exacta using this model